In [1]:
import pandas as pd
import numpy as np
import cplex

from sys import path
path.append("..") 

from CPNorm import CorrNorm

In [10]:
df = pd.DataFrame({
    'A': [2000,3000,4000, 8000],
    'B': [100,55,50, 10]
}, index=['cell1', 'cell2', 'cell3', 'cell4'])
normalization = CorrNorm(df, 0.001)
normalization.write_problem()
normalization.solve()
result = normalization.get_beta_values()
covThres = normalization.get_covariance_thresholds()


[3.322219294733919, 3.485011214578573, 3.6074550232146687, 3.9036325160842376]


Default row names c1, c2 ... being created.


Version identifier: 20.1.0.1 | 2022-01-05 | 9df5e5246
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Aggregator did 1 substitutions.
Reduced QCP has 37 rows, 34 columns, and 83 nonzeros.
Reduced QCP has 5 quadratic constraints.
Presolve time = 0.00 sec. (0.02 ticks)
Parallel mode: using up to 12 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 476
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.02 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 12
  Rows in Factor            = 37
  Integer space required    = 60
  Total non-zeros in factor = 525
  Total FP ops to factor    = 10009
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf Inf Ratio
   0   1.8284271e+00  -1.0000000e+00  6.07e+00  0.00e+00  3.15e+01  1.00e+00
   1   1.8517650e-02  -3.5527598e-01  6.07e+00  0.00e+00  3.15e+01  4.02e+01
   2  -5.2714084e-02  -1.5682103e-01  6.32e-01  0.00e+00  3.2

In [11]:
print(result)
print([10**x for x in result])
print(covThres)

print(normalization.get_librarySize_variance())

[-0.25648024321058954, -0.0942574335880967, 0.02790794934824699, 0.3228297274505718]
[0.5540127463407776, 0.8049011843296215, 1.0663700753834744, 2.10295377926494]
[-0.48643241168161644, 0.48643241168138646]
cell1    3790.526507
cell2    3795.496962
cell3    3797.931031
cell4    3808.928222
dtype: float64


In [9]:
df = pd.read_csv("/Users/t.stohn/Desktop/Projects/scNormalization/CorrelationPreservationNorm/test/LinearTrajectory/Simulation.tsv", sep = "\t")
df = df[["sample_id","ab_id","ab_count_normalized"]]
df = df.pivot(index="sample_id", columns="ab_id", values="ab_count_normalized")

df

ab_id,AB0,AB1,AB10,AB11,AB12,AB13,AB14,AB15,AB16,AB17,AB18,AB19,AB2,AB3,AB4,AB5,AB6,AB7,AB8,AB9
sample_id,,,,,,,,,,,,,,,,,,,,
sample_1,1766556.0,9145607.0,3551513.0,5115651.0,2447416.0,2539424.0,10783352.0,4361185.0,3036268.0,12531506.0,2778645.0,2686637.0,2852252.0,6274954.0,14518881.0,3349096.0,2981063.0,2134588.0,18714452.0,18677648.0
sample_10,893919.0,7065638.0,3049123.0,2669513.0,1787839.0,2253167.0,3575678.0,5498217.0,5461481.0,4800225.0,1947030.0,2522567.0,2473585.0,6453364.0,10420897.0,1077602.0,1126583.0,685746.0,3379750.0,13874120.0
sample_100,2920268.0,4772676.0,3988127.0,2985647.0,3465094.0,1656271.0,4903435.0,6908395.0,8869769.0,39576167.0,2440821.0,2571579.0,6341776.0,5775156.0,4031713.0,2724130.0,1852409.0,5186744.0,19875255.0,8782596.0
sample_101,1820762.0,8596763.0,5139620.0,4955239.0,2719619.0,6522477.0,3825905.0,6199810.0,4816953.0,31206479.0,5324001.0,3503238.0,3572381.0,6130667.0,27080955.0,3618477.0,4079429.0,1843810.0,30215431.0,29086098.0
sample_102,6532003.0,10802928.0,17963008.0,25813973.0,19784432.0,6720426.0,15576315.0,27509782.0,9107120.0,393553178.0,12749967.0,6532003.0,7034465.0,19847240.0,54579909.0,7348503.0,8730273.0,15073853.0,71977648.0,82466537.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sample_95,1304789.0,4056907.0,2214852.0,4572244.0,4648996.0,2291604.0,5679669.0,8201530.0,3530605.0,17861354.0,2817906.0,1853020.0,1129355.0,4824430.0,4287164.0,1798197.0,921027.0,910063.0,14275926.0,8782655.0
sample_96,878673.0,6168284.0,3268663.0,6572474.0,3497118.0,4200057.0,3092929.0,5922256.0,3479545.0,14919867.0,1581611.0,878673.0,1441024.0,8364967.0,2864474.0,3883734.0,2056095.0,1089554.0,11967526.0,4182483.0
sample_97,6947034.0,19274102.0,16087868.0,10603367.0,18751768.0,9610934.0,9402001.0,8305101.0,9349767.0,68530139.0,3134000.0,6685867.0,9297534.0,20893335.0,40010736.0,4178667.0,7939467.0,5171100.0,50248471.0,83573340.0


In [10]:
#eta towards 1 results in covarainces going to zero
#eta towards 0 is only library size normalisation
normalization = CorrNorm(df, 0.999999)
normalization.write_problem()
normalization.solve()
result = normalization.get_result()

sample_id
sample_1      130246694.0
sample_10      81016044.0
sample_100    139628028.0
sample_101    190258114.0
sample_102    819703563.0
                 ...     
sample_95      95962293.0
sample_96      90310007.0
sample_97     407994598.0
sample_98     279015084.0
sample_99     264517539.0
Length: 500, dtype: float64
[8.114766708587464, 7.908571032837742, 8.144972604418612, 8.279343187336346, 8.913656822835645, 8.545185384356953, 8.542838710691823, 8.298873492135256, 8.40684743980428, 8.368557744378858, 8.215402475928183, 7.704893605763836, 8.28696551519093, 8.24895202667026, 8.302565936178228, 8.434925029543392, 8.089455659567957, 8.429069882454504, 7.881108006981323, 8.253316239162693, 8.200490620281192, 7.9573699882825295, 8.559915131381915, 8.036619071733412, 8.38406608789022, 8.32718565822216, 8.227778022313222, 7.7760932402461425, 8.223628732220751, 7.575093768402287, 7.9364014950208315, 8.205423862741934, 7.951693703058689, 7.846818676827836, 7.974033406181849, 8.0273944132

Default row names c1, c2 ... being created.


Version identifier: 20.1.0.1 | 2022-01-05 | 9df5e5246
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Aggregator did 1 substitutions.
Reduced QCP has 21085 rows, 20586 columns, and 61667 nonzeros.
Reduced QCP has 41 quadratic constraints.
Presolve time = 0.02 sec. (13.14 ticks)
Parallel mode: using up to 12 threads for barrier.

***NOTE: Found 41 dense columns.

Number of nonzeros in lower triangle of A*A' = 1292360
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.03 sec. (20.31 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 12
  Rows in Factor            = 21085
  Integer space required    = 41584
  Total non-zeros in factor = 1313445
  Total FP ops to factor    = 85141245
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf Inf Ratio
   0   1.8284271e+00  -1.0000000e+00  7.39e+02  0.00e+00  1.06e+03  1.00e+00
   1   2.3150491e+00   1.6478187e+00  7.39e+02  0.00e+00  1.06e+03  2.17e+

AttributeError: 'CorrNorm' object has no attribute 'get_result'

In [12]:
print(result)

[-0.01198830706138512, 0.05897070201959319, 0.04282759468156962, -0.05924845266329246, -0.05934185409588268, -0.004288799516309242, 0.02985114547000253, 0.05659755835773126, 0.022158231681203185, -0.019721573607799258, -0.04112617577349143, -0.12574864754372478, 0.040914385594122596, -0.009657408720508166, 0.06925753873255568, -0.030857308373357838, -0.019956827320684496, -0.02246764736123001, -0.030646442441784365, 0.01949192337307065, 0.07577089902986417, 0.021373472822173504, -0.013394461425578865, 0.0314956965561872, 0.031284456278437484, -0.007220786857833182, 0.044775246060409585, -0.023938660113468624, -0.04073738669744336, -0.01322923562142894, 0.021673585589714113, -0.00013435284887736786, -0.01620257826272918, -0.017055029659032724, -0.008338658103655647, 0.03530347413119829, 0.02067766143171721, 0.02043712872466093, 0.036802471417388974, -0.04492775400786221, -0.01152802552183541, 0.061603837502921095, -0.02396326265635947, -0.011741396105643643, -0.07786501605717364, 0.0125